# [Training your own Chatbot using GPT​](https://www.youtube.com/watch?v=DxygPxcfW_I)

# [Transformers](https://pypi.org/project/transformers/)
**Transformers** provides thousands of pretrained models to perform tasks on different modalities such as text, vision, and audio.

These models can be applied on:

📝 **Text**, for tasks like text classification, information extraction, question answering, summarization, translation, and text generation, in over 100 languages.

🖼️ **Images**, for tasks like image classification, object detection, and segmentation.

🗣️ **Audio**, for tasks like speech recognition and audio classification.

In [1]:
!pip install transformers

# [torch 2.3.0](https://pypi.org/project/torch/)
**PyTorch** is a Python package that provides two high-level features:

* Tensor computation (like NumPy) with strong GPU acceleration
* Deep neural networks built on a tape-based autograd system

In [2]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

# [python-docx](https://pypi.org/project/python-docx/)
**python-docx** is a Python library for reading, creating, and updating Microsoft Word 2007+ (.docx) files.

In [3]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.4 MB/s eta 0:00:00


# [PyPDF2](https://pypi.org/project/PyPDF2/)
**PyPDF2** is a free and open-source pure-python PDF library capable of splitting, merging, cropping, and transforming the pages of PDF files. It can also add custom data, viewing options, and passwords to PDF files. PyPDF2 can retrieve text and metadata from PDFs as well.

In [4]:
!pip install -U PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


### Import general required libraries

In [5]:
import os
import re
from PyPDF2 import PdfReader
import docx
import torch

# All the below classes are provided by the "Hugging Face" Transformers library.

## [class transformers.GPT2Tokenizer](https://huggingface.co/docs/transformers/en/model_doc/gpt2#transformers.GPT2Tokenizer)
* The `GPT2Tokenizer` class is used for tokenizing text data in a way that is compatible with the GPT-2 model.
* Tokenization involves converting a string of text into a list of tokens (words or subwords) that can be processed by the model.
* This step is crucial for preparing text data for tasks such as text generation, text completion, or any other NLP application involving GPT-2.

## [class transformers.GPT2LMHeadModel](https://huggingface.co/docs/transformers/en/model_doc/gpt2#transformers.GPT2LMHeadModel)

`GPT2LMHeadModel` stands for "Language Modeling Head." This model includes a language modeling head on top of the GPT-2 architecture, which is specifically designed for generating text. It predicts the next token in a sequence, making it suitable for tasks like text completion, text generation, and other sequence prediction tasks.

## TextDataset
The `TextDataset` class helps in loading and processing text data in a format suitable for training and evaluation with transformer models.

1. **Tokenization**: Automatically tokenizes the input text using a specified tokenizer.
2. **Encoding**: Converts the tokenized text into numerical format that models can process.
3. **Handling Large Datasets**: Efficiently handles large text files by loading data in chunks.


## [class transformers.DataCollatorForLanguageModeling](https://huggingface.co/docs/transformers/v4.41.0/en/main_classes/data_collator#transformers.DataCollatorForLanguageModeling)

This collator handles the batching and padding of sequences to ensure that all input sequences in a batch have the same length, which is necessary for efficient GPU utilization.

It can be used for both <u>causal language modeling</u> (**CLM**) and <u>masked language modeling</u> (**MLM**) tasks, depending on the configuration.

In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# <font color='red'>Caution: Use only single GPT model at a time.</font>

# [GPT-2](https://huggingface.co/openai-community/gpt2)
Pretrained model on English language using a <u>causal language modeling (CLM)</u> objective.

## Model description
**GPT-2** is a transformers model pretrained on a very large corpus of English data in a self-supervised fashion. This means it was pretrained on the raw texts only, with no humans labelling them in any way (which is why it can use lots of publicly available data) with an automatic process to generate inputs and labels from those texts. More precisely, it was trained to guess the next word in sentences.

This is the <u>**smallest** version of GPT-2, with **124M**</u> parameters.

In [6]:
'''
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="openai-community/gpt2")
'''

'\n# Use a pipeline as a high-level helper\nfrom transformers import pipeline\n\npipe = pipeline("text-generation", model="openai-community/gpt2")\n'

In [7]:
'''
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
'''

'\n# Load model directly\nfrom transformers import AutoTokenizer, AutoModelForCausalLM\n\ntokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")\nmodel = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")\n'

# [GPT-2 Medium](https://huggingface.co/openai-community/gpt2-medium)
## Model Description
**GPT-2 Medium** is the **355M** parameter version of GPT-2, a transformer-based language model created and released by OpenAI. The model is a pretrained model on English language using a <u>causal language modeling (CLM)</u> objective.

In [8]:
'''
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="openai-community/gpt2-medium")
'''

'\n# Use a pipeline as a high-level helper\nfrom transformers import pipeline\n\npipe = pipeline("text-generation", model="openai-community/gpt2-medium")\n'

In [9]:
'''
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-xl")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-xl")
'''

'\n# Load model directly\nfrom transformers import AutoTokenizer, AutoModelForCausalLM\n\ntokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-xl")\nmodel = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-xl")\n'

# [GPT-2 Large](https://huggingface.co/openai-community/gpt2-large)
## Model Description
**GPT-2 Large** is the **774M** parameter version of GPT-2, a transformer-based language model created and released by OpenAI. The model is a pretrained model on English language using a <u>causal language modeling (CLM)</u> objective.

In [10]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="openai-community/gpt2-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-large")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-large")

# [GPT-2 XL](https://huggingface.co/openai-community/gpt2-xl)
## Model Description
**GPT-2 XL** is the **1.5B** parameter version of GPT-2, a transformer-based language model created and released by OpenAI. The model is a pretrained model on English language using a <u>causal language modeling (CLM)</u> objective.

In [12]:
'''
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2-xl')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)
'''

'\nfrom transformers import pipeline, set_seed\ngenerator = pipeline(\'text-generation\', model=\'gpt2-xl\')\nset_seed(42)\ngenerator("Hello, I\'m a language model,", max_length=30, num_return_sequences=5)\n'